In [1]:
import json

from crypto_chatter.config import load_default_data_config, load_default_graph_config
from crypto_chatter.config.path import BASE_DIR
from crypto_chatter.data import CryptoChatterData

dataset = 'twitter:blockchain-interoperability-attacks'
graph_type = 'tweet'
data_config = load_default_data_config(dataset)
graph_config = load_default_graph_config(dataset, graph_type)

init_clusters = json.load(open(data_config.data_dir / 'kmeans_init_clusters.json'))

old_kmeans_dir = BASE_DIR / 'old/analysis-data/kmeans_clusters_resampled'
resample_clusters = {
    i: json.load(open(old_kmeans_dir / f'{i}_ids.json'))
    for i in range(6)
}

data = CryptoChatterData(data_config)


we are using cpu 🐌🐌🐌


Output()

Loaded cache in 116 seconds


In [3]:
data.fit_tfidf()

In [15]:
import numpy as np
from crypto_chatter.data.utils import is_spam,preprocess_text

random_size = 1000000
text = data[data.data_config.text_col].values
rng = np.random.RandomState(0)
# first filter out spam
not_spam = [t for t in text if not is_spam(t)]
# Then get random indices
random_idxs = rng.permutation(np.arange(len(not_spam)))[:random_size]
subset = [preprocess_text(not_spam[i]) for i in random_idxs]

top_n = 10
terms = data.tfidf.get_feature_names_out()
vecs = data.tfidf.transform(subset)
tfidf_scores = vecs.toarray().sum(0)
sorted_idxs = tfidf_scores.argsort()[::-1]
keywords = terms[sorted_idxs][:top_n]
keyword_scores = tfidf_scores[sorted_idxs][:top_n]
keywords_with_score = dict(zip(keywords, keyword_scores))
print(keywords)

['user' 'http' 'pump' 'just' 'signal' 'happen' 'crypto' 'wallstreetbets'
 'event' 'kucoin']


[autoreload of crypto_chatter.data.tfidf failed: Traceback (most recent call last):
  File "/Users/inwon/miniconda3/envs/twitter/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/inwon/miniconda3/envs/twitter/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/Users/inwon/miniconda3/envs/twitter/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/inwon/Documents/research/blockchain-social-media/crypto_chatter/da

In [10]:
import pickle

old_ids = pickle.load(open('/Users/inwon/Downloads/id.pkl','rb'))

In [14]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from crypto_chatter.data.utils import is_spam, preprocess_text

stop_words = list(ENGLISH_STOP_WORDS | set(['https', '@']))

rng = np.random.RandomState(0)
for cluster_id, cluster in init_clusters.items():
    ids = [old_ids[i] for i in cluster]
    text = [
        preprocess_text(t)
        for t in data[data.data_config.text_col][data.df['id'].isin(ids)].values
        if not is_spam(t)
    ]

    sampled_text = ([
        text[i]
        for i in rng.permutation(len(text))[:100000]
    ])
    top_n = 10

    terms = data.tfidf.get_feature_names_out()
    vecs = data.tfidf.transform(sampled_text)
    tfidf_scores = vecs.toarray().sum(0)
    sorted_idxs = tfidf_scores.argsort()[::-1]
    keywords = terms[sorted_idxs][:top_n]
    keyword_scores = tfidf_scores[sorted_idxs][:top_n]
    keywords_with_score = dict(zip(keywords, keyword_scores))
    print('Cluster', cluster_id)
    print(keywords)

    tfidf = TfidfVectorizer(
        stop_words=stop_words,
        # ngram_range=ngram_range,
        # max_df=max_df,
        # min_df=min_df,
        # max_features=max_features,
    )

    vecs = tfidf.fit_transform(sampled_text)
    terms = tfidf.get_feature_names_out()
    sorted_idxs = tfidf_scores.argsort()[::-1]
    keywords = terms[sorted_idxs][:top_n]
    keyword_scores = tfidf_scores[sorted_idxs][:top_n]
    keywords_with_score = dict(zip(keywords, keyword_scores))
    print('V2')
    print(keywords)
    
    print('='*80)
    # break

Cluster 0
['user' 'pump' 'http' 'signal' 'just' 'event' 'happen' 'wallstreetbets'
 'kucoin' 'big']
V2
['birdie' 'asinine' 'actors' 'bakht_md1' 'agulu' '80x5imzeqg'
 'abvp68thconfjaipur' 'bkexglobal' 'aimkellar' '2437']
Cluster 1
['user' 'crypto' 'http' 'promote' 'roll' 'token' 'price' '000' 'binance'
 'security']
V2
['341800' '07477579' '12cq' '2100522' '2461' '3330' '206086' '000' '03267'
 '2724']
Cluster 2
['user' 'http' 'crypto' 'v2' 'rollup' 'address' 'tokens' 'claiming'
 'compatible' 'evm']
V2
['anticipated' '50mp' '22pm' 'antivaxers' 'admission'
 '0x3556c6c04be64e14e750e99dc56f69592b0a51de' 'amir84186215' '1999' '1x'
 '353take']
Cluster 3
['roll' 'crypto' 'sushi' 'user' 'project' 'security' 'good' 'try' 'http'
 'bridge']
V2
['059m' '00002170' '0924' '1208' '036100000' '0722' '00039049' '11245'
 '0008312' '000005370']
Cluster 4
['user' 'http' 'pump' 'crypto' 'just' 'signal' 'kucoin' 'happen' 'event'
 'wallstreetbets']
V2
['ax9qwwe5qk' '81' 'alpi' '360k' '9jncnpuv8q' 'apys' '_angel